### import library neeeded

In [687]:
import pandas as pd
import numpy as np

In [688]:
def create_key_personal(dataframe):
    dataframe["key_personal"] = (
        dataframe["district"].astype(str) +
        dataframe["commune"].astype(str) +
        dataframe["enumeration_area"].astype(str) +
        dataframe["household_id"].astype(str) + 
        dataframe["person_code"].astype(str)
        )
    return dataframe

### Dimension DanToc and Province

In [689]:
df_dim_ethnicity = pd.read_csv("data/dimension/dim_ethnicity.csv")
df_dim_province = pd.read_csv("data/dimension/dim_province.csv")
df_dim_scaling_education = pd.read_csv("data/dimension/dim_general_education_scaling.csv")

## Merge into master data

### import needed datasets

In [690]:
# person records
df_personal_information = pd.read_csv("data/classify/cleaned/1/1_members_family_information.csv")
# person records
df_education = pd.read_csv("data/classify/cleaned/2/2_education.csv")
# person records
df_employment = pd.read_csv("data/classify/cleaned/4/4_employments_wages_and_earnings.csv")
# household records
df_personal_information_more = pd.read_csv("data/classify/cleaned/1/ho1_personal_information.csv")
# hoysehold records
df_expenditure = pd.read_csv("data/classify/cleaned/5/ho4_totalcost.csv")

C:\Users\admin\AppData\Local\Temp\ipykernel_20848\1003434308.py:6: DtypeWarning: Columns (0: working_hours) have mixed types. Specify dtype option on import or set low_memory=False.
  df_employment = pd.read_csv("data/classify/cleaned/4/4_employments_wages_and_earnings.csv")


### adding to master data

### create key_personal

In [691]:
df_personal_information = create_key_personal(df_personal_information)
df_education = create_key_personal(df_education)
df_employment = create_key_personal(df_employment)

### join data - key_personal

In [692]:
# this is for the ignore columns similarity
key_exclude = ["province", "district", "commune","enumeration_area","household_id","person_code"]

In [693]:
df_master_data_inc_education = df_personal_information.merge(df_education.loc[:, ~df_education.columns.isin(key_exclude)], on=["key_household","key_personal"],how="left")

In [694]:
df_master_data_inc_employment = df_master_data_inc_education.merge(df_employment.loc[:, ~df_employment.columns.isin(key_exclude)], on=["key_household","key_personal"], how="left")

### join data - key_household

In [695]:
### duplicated drop - personal_information_more - ho1
df_personal_information_more_drop_duplicate = df_personal_information_more.drop_duplicates()

In [696]:
df_master_data_inc_information_more = df_master_data_inc_employment.merge(df_personal_information_more_drop_duplicate.loc[:, ~df_personal_information_more_drop_duplicate.columns.isin(key_exclude)], on=["key_household"], how="left")

In [697]:
### duplicated drop - total_cost - ho4
df_expenditure_drop_duplicate = df_expenditure.drop_duplicates()

In [698]:
df_master_data_inc_expenditure = df_master_data_inc_information_more.merge(df_expenditure_drop_duplicate.loc[:,~df_expenditure_drop_duplicate.columns.isin(key_exclude)], on="key_household", how="left")

### filtered columns neeeded

In [699]:
columns_extract = ["key_household", "key_personal", "province", "district", "commune", "enumeration_area", "household_id", "person_code", "relationship_to_head", "gender", "age", "age_square", "mother_proxy", "mother_proxy_child_below_6","has_job", "working_hours", "highest_degree_general_education", "highest_degree_vocational_training", "marital_status", "ethnicity", "household_size", "total_cost_yearly_food_and_beverages_expenditures_during_holidays_and_fesitval","total_cost_monthly_regular_food_and_beverages_expenditures", "total_cost_monthly_for_daily_spending_exclude_food", "total_cost_yearly_spending_exclude_food", "total_cost_others_expenditures_included_in_total_expenditure"]

In [700]:
# "Khác" value in highest_degree_general_education -> will exist in avocation training -> trường nghề từ sơ cấp -> trung cấp - và có không đi học 
# transform and replace data "Khác" cột highest_degree_general_education = value của cột highest_degree_vocational_training
# na at highest_degree_general_education will also na at highest_degree_vocational_training
df_master_data_filtered = df_master_data_inc_expenditure.loc[:,df_master_data_inc_expenditure.columns.isin(columns_extract)]
df_master_data_filtered["highest_degree_general_education"] = np.where(
    df_master_data_filtered["highest_degree_general_education"] == "Khác",
    df_master_data_filtered["highest_degree_vocational_training"],
    df_master_data_filtered["highest_degree_general_education"]
)

### calculate total_cost per person

In [701]:
df_master_data_filtered["total_cost_yearly"] = (
    df_master_data_filtered["total_cost_yearly_food_and_beverages_expenditures_during_holidays_and_fesitval"] 
    + 12*(df_master_data_filtered["total_cost_monthly_regular_food_and_beverages_expenditures"] + df_master_data_filtered["total_cost_monthly_for_daily_spending_exclude_food"])
    + df_master_data_filtered["total_cost_yearly_spending_exclude_food"]
    + df_master_data_filtered["total_cost_others_expenditures_included_in_total_expenditure"]
)

In [702]:
df_master_data_filtered["total_cost_yearly_per_person"] = df_master_data_filtered["total_cost_yearly"] / df_master_data_filtered["household_size"]

### filter layer aggregation 

In [703]:
columns_aggr = ["key_household", "key_personal", "province", "district", "commune", "enumeration_area", "household_id", "person_code", "relationship_to_head", "gender", "age", "age_square", "marital_status", "ethnicity", "mother_proxy", "mother_proxy_child_below_6","has_job", "working_hours", "highest_degree_general_education", "household_size", "total_cost_yearly", "total_cost_yearly_per_person"]

In [704]:
df_master_data_layer_aggr = df_master_data_filtered.loc[:,df_master_data_filtered.columns.isin(columns_aggr)]

### mapping number_province + region, mapping ethnicity_number -> ethnicity name

In [705]:
df_master_get_number_province = df_master_data_layer_aggr.merge(df_dim_province, on = ["province"], how="left")
df_master_get_name_ethinicty = df_master_get_number_province.merge(df_dim_ethnicity, on = ["ethnicity"], how = "left")

### scaling education

In [706]:
df_master_scaling_education = df_master_get_name_ethinicty.merge(df_dim_scaling_education, on = ["highest_degree_general_education"], how ='left')

### group identification - Group 1 | Group 2

In [707]:
# "Full data: 18-50
# Group 1: age from 18-35
# Group 2: age from 36-50"
df_master_scaling_education["group_1"] = (
    (df_master_scaling_education["age"] >= 18) &
    (df_master_scaling_education["age"] <= 35)
)
df_master_scaling_education["group_2"] = (
    (df_master_scaling_education["age"] >= 36) &
    (df_master_scaling_education["age"] <= 45)
)

### remove duplicated key_personal

In [708]:
df_master_scaling_education_remove_dup = df_master_scaling_education.drop_duplicates()

### number of mother | number of children per household

In [709]:
df_master_scaling_education_remove_dup["number_of_mothers_of_household"] = (
    df_master_scaling_education_remove_dup
    .groupby("key_household")["mother_proxy"]
    .transform("sum")
)


In [710]:
# household counts
count_grandchild = (
    df_master_scaling_education_remove_dup["relationship_to_head"].eq("Cháu nội ngoại")
    .groupby(df_master_scaling_education_remove_dup["key_household"])
    .transform("sum")
)

count_child = (
    df_master_scaling_education_remove_dup["relationship_to_head"].eq("Con")
    .groupby(df_master_scaling_education_remove_dup["key_household"])
    .transform("sum")
)

# condition count children base on mother-proxy
mother_is_con = (
    (df_master_scaling_education_remove_dup["relationship_to_head"] == "Con") &
    (df_master_scaling_education_remove_dup["mother_proxy"])
)

mother_is_head_or_wife = (
    df_master_scaling_education_remove_dup["mother_proxy"] &
    df_master_scaling_education_remove_dup["relationship_to_head"].isin(["Chủ hộ","Vợ/chồng"])
)

# final count children
df_master_scaling_education_remove_dup["children_count_based_on_mother_proxy"] = np.select(
    [mother_is_con, mother_is_head_or_wife],
    [count_grandchild, count_child],
    default=np.nan
)


### filter column need for final master data

In [711]:
columns_used_for_processing_p2 = ['key_household', 'key_personal', 'num_province','region','district', 'commune', 'enumeration_area',
       'household_id', 'person_code', 'gender','age', 'age_square', 'mother_proxy',
       'mother_proxy_child_below_6','has_job','working_hours','scaling_education_compare','marital_status', 'ethnicity',
       'total_cost_yearly', 'total_cost_yearly_per_person','household_size',
        'group_1', 'group_2',
       'number_of_mothers_of_household',
       'children_count_based_on_mother_proxy'
       ]

In [712]:
df_master_data_finalize = df_master_scaling_education_remove_dup[columns_used_for_processing_p2]

### Datatype Scaling -> normalize

### has_job column

In [713]:
mapping_has_job = {'Có': True, 'Không': False}
df_master_data_finalize['has_job']=df_master_data_finalize['has_job'].map(mapping_has_job)

### data type normalize

In [714]:
df_master_data_finalize.info()

<class 'pandas.DataFrame'>
Index: 263706 entries, 0 to 264337
Data columns (total 26 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   key_household                         263706 non-null  int64  
 1   key_personal                          263706 non-null  str    
 2   num_province                          263706 non-null  int64  
 3   region                                263706 non-null  str    
 4   district                              263706 non-null  int64  
 5   commune                               263706 non-null  int64  
 6   enumeration_area                      263706 non-null  int64  
 7   household_id                          263706 non-null  int64  
 8   person_code                           263706 non-null  int64  
 9   gender                                263706 non-null  str    
 10  age                                   263706 non-null  int64  
 11  age_square      

In [715]:
# columns datatype classification
int_columns = ["key_household", "key_personal", "num_province", "district", "ethnicity", "commune","enumeration_area", "household_id","person_code","age","age_square","scaling_education_compare", "household_size","number_of_mothers_of_household","children_count_based_on_mother_proxy"]
str_columns = ["region", "gender","has_job","marital_status"]
float_columns = ["total_cost_yearly", "total_cost_yearly_per_person","working_hours"]
bool_columns = ["mother_proxy","mother_proxy_child_below_6","group_1","group_2"]

In [716]:
df_master_data_finalize_dtype = df_master_data_finalize.copy()

df_master_data_finalize_dtype[int_columns] = df_master_data_finalize_dtype[int_columns].apply(pd.to_numeric, errors="coerce").astype("Int64")
df_master_data_finalize_dtype[float_columns] = df_master_data_finalize_dtype[float_columns].apply(pd.to_numeric, errors="coerce")
df_master_data_finalize_dtype[str_columns] = df_master_data_finalize_dtype[str_columns].astype("string")
df_master_data_finalize_dtype[bool_columns] = df_master_data_finalize_dtype[bool_columns].astype("boolean")


In [717]:
# change bool_columns from True/False to 1/0
for col in bool_columns:
    df_master_data_finalize_dtype[col] = np.where(
        df_master_data_finalize_dtype[col] == True, 
        1, 
        0
    )

### gender and marital_status normalize

In [718]:
mapping_gender = {"Nam":"Male", "Nữ":"Female"}
df_master_data_finalize_dtype["gender"] = df_master_data_finalize_dtype["gender"].map(mapping_gender)

In [719]:
marital_status_dict = {
    "Đang có vợ/chồng": "Married",
    "Chưa vợ/chồng": "Never married",
    "Goá": "Widowed",
    "Ly hôn": "Divorced",
    "Ly thân": "Separated",
    "NR": "No response",
    pd.NA: pd.NA
}
df_master_data_finalize_dtype["marital_status"] = df_master_data_finalize_dtype["marital_status"].map(marital_status_dict)


In [720]:
region_dict = {
    "Đồng bằng sông Cửu Long": "Mekong River Delta",
    "Trung du và miền núi phía Bắc": "Northern Midlands and Mountain Areas",
    "Đồng bằng sông Hồng": "Red River Delta",
    "Bắc Trung Bộ và duyên hải miền Trung": "North Central and Central Coastal Areas",
    "Tây Nguyên": "Central Highlands",
    "Đông Nam Bộ": "South East",
    pd.NA: pd.NA
}

df_master_data_finalize_dtype["region"] = df_master_data_finalize_dtype["region"].map(region_dict)

### remove complex household

In [721]:
# before remove: 263,706 person records

# after remove: 230,188 person records 
df_master_data_finalize_dtype_rm_complex_household = (
    df_master_data_finalize_dtype
    .groupby("key_household")
    .filter(
        lambda g: not (
            (g["number_of_mothers_of_household"] >
             g["children_count_based_on_mother_proxy"])
            & g["children_count_based_on_mother_proxy"].notna()
        ).any()
    )
)


### adding column "has_married"

In [722]:
df_master_data_finalize_dtype_rm_complex_household["has_married"] = np.where(df_master_data_finalize_dtype_rm_complex_household["marital_status"]=="Married",1,0)

### filter gender:Female and Age:from 18-50

In [723]:
# filter gender: "Female" only: 115,430 person records
# filter gender: "Female" and age(18,50) only: 50,908 person records 
# filter gender: "Female" and age(18,45) only: 42,318 person records --na: 14,964 person_records
# filter gender: "Female" age age(18,42) only: 36,903 person records --na: 13,141 person_records
df_master_data_finalize_dtype_rm_complex_household = (
    df_master_data_finalize_dtype_rm_complex_household[
        (df_master_data_finalize_dtype_rm_complex_household["gender"] == "Female") &
        (df_master_data_finalize_dtype_rm_complex_household["age"].between(18, 45))
    ]
)

In [724]:
df_master_data_finalize_dtype_rm_complex_household.info()

<class 'pandas.DataFrame'>
Index: 42318 entries, 6 to 264329
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   key_household                         42318 non-null  Int64  
 1   key_personal                          42318 non-null  Int64  
 2   num_province                          42318 non-null  Int64  
 3   region                                42318 non-null  str    
 4   district                              42318 non-null  Int64  
 5   commune                               42318 non-null  Int64  
 6   enumeration_area                      42318 non-null  Int64  
 7   household_id                          42318 non-null  Int64  
 8   person_code                           42318 non-null  Int64  
 9   gender                                42318 non-null  str    
 10  age                                   42318 non-null  Int64  
 11  age_square                    

## Allocate to folder data/classify/cleaned/data_training

### full data

In [725]:
df_master_data_finalize_dtype_rm_complex_household.to_csv("data/classify/cleaned/data_training/full_data.csv",index=False)

### group 1

In [726]:
df_group_1 = df_master_data_finalize_dtype_rm_complex_household[df_master_data_finalize_dtype_rm_complex_household["group_1"] == 1]
df_group_1.to_csv("data/classify/cleaned/data_training/group1.csv",index=False)

### group 2

In [727]:
df_group_2 = df_master_data_finalize_dtype_rm_complex_household[df_master_data_finalize_dtype_rm_complex_household["group_2"] == 1]
df_group_2.to_csv("data/classify/cleaned/data_training/group2.csv",index=False)